In [0]:
-- CONTRACT ANALYSIS
-- -- Parse Data

CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.parsed') (
  path STRING NOT NULL PRIMARY KEY,
  parsed VARIANT
);

-- merge into prevents old files from being parsed again
-- This takes quite a while for a large number of documents (more pages = more time)
-- We aren't describing figures or illustrations to save time, nor saving images
MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.parsed') AS target
USING (
  SELECT 
    bytes.path,
    AI_PARSE_DOCUMENT(bytes.content) AS parsed
  FROM IDENTIFIER(:catalog || '.' || :schema || '.bytes') AS bytes
  LEFT JOIN IDENTIFIER(:catalog || '.' || :schema || '.parsed') AS parsed
    ON bytes.path = parsed.path
  WHERE bytes.file_name RLIKE '\\.(pdf|jpg|jpeg|png|doc|docx|ppt|pptx)$'
    AND parsed.path IS NULL
  LIMIT :batch_size
) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN
  INSERT *